# Group 7 - Models For Leather Defect Detection

#### Libraries Used

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import ssl


In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

#### Call backs

In [4]:
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                         patience=5, min_lr=0.0001)


model_checkpoint = ModelCheckpoint(
    filepath='/Users/manuelmena/Documents/IE/Third Year/SEM 2 Junior/Machine Learning 2/Project/model-{epoch:02d}-{val_loss:.2f}.keras', 
    save_best_only=True,
    monitor='val_loss', 
    mode='min',  
    verbose=1 
)


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5, 
    restore_best_weights=True, 
    verbose=1 
)

## Data Preprocessing and Loading

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

data = '/Users/manuelmena/Desktop/Leather Defect Classification'
train_generator = train_datagen.flow_from_directory(
    data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', 
    subset='training' 
)

validation_generator = train_datagen.flow_from_directory(
    data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


## ResNet Model

In [ ]:

base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),  
    layers.Dropout(0.3), 
    layers.Dense(512, activation='relu'), 
    layers.Dropout(0.3), 
    layers.Dense(6, activation='softmax') 
])

resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_resnet = resnet.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=validation_generator.n // validation_generator.batch_size
    )


plt.plot(history_resnet.history['accuracy'], label='ResNet Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()

### Resnet FineTuned

In [ ]:

base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


for layer in base_model.layers[:-10]:
    layer.trainable = False


resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),  
    layers.Dropout(0.3), 
    layers.Dense(512, activation='relu'), 
    layers.Dropout(0.3), 
    layers.Dense(6, activation='softmax') 
])

resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_resnet = resnet.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    validation_steps=validation_generator.n // validation_generator.batch_size
)

plt.plot(history_resnet.history['accuracy'], label='ResNet Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()

## Efficient Net 

In [ ]:

base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False

efficientnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),  
    layers.Dropout(0.3), 
    layers.Dense(512, activation='relu'), 
    layers.Dropout(0.3), 
    layers.Dense(6, activation='softmax') 
])

efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_efficientnet = efficientnet.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    validation_steps=validation_generator.n // validation_generator.batch_size
)


plt.plot(history_efficientnet.history['accuracy'], label='EfficientNet Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()


### Efficient Fine Tuned

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
for layer in base_model.layers[:-10]:
    layer.trainable = False

efficientnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),  
    layers.Dropout(0.3), 
    layers.Dense(512, activation='relu'), 
    layers.Dropout(0.3), 
    layers.Dense(6, activation='softmax') 
])

efficientnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_efficientnet = efficientnet.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    validation_steps=validation_generator.n // validation_generator.batch_size
)


plt.plot(history_efficientnet.history['accuracy'], label='EfficientNet Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()

## VGG Fine Tuned

In [ ]:

base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

for layer in base_model.layers[:15]:
    layer.trainable = False

vgg = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),  
    layers.Dropout(0.3), 
    layers.Dense(512, activation='relu'), 
    layers.Dropout(0.3), 
    layers.Dense(6, activation='softmax') 
])



vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg.summary()

history_vgg = vgg.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    validation_steps=validation_generator.n // validation_generator.batch_size
)


plt.plot(history_vgg.history['accuracy'], label='VGG Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()

## Our CNN Model

In [ ]:
cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding = 'valid'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax')
])

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_cnn = cnn.fit(
    train_generator, 
    epochs=10, 
    validation_data=validation_generator,
    callbacks=[reduce_lr_on_plateau, model_checkpoint, early_stopping],
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    validation_steps=validation_generator.n // validation_generator.batch_size
)


plt.plot(history_cnn.history['accuracy'], label='CNN Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Comparison - Accuracy')
plt.legend()
plt.show()
